In [ ]:
import matplotlib.pyplot as plt
import mpl_lego as mplego
import numpy as np
import pandas as pd
import pickle

from mpl_lego.labels import bold_text
from pyprojroot import here

from normative_evaluation_llms_everyday_dilemmas import keys

In [ ]:
mplego.style.use_latex_style()

In [ ]:
df = pd.read_csv(here('data/normative_evaluation_everyday_dilemmas_dataset.csv'))

In [ ]:
moral_labels_df = df[keys.MORAL_LABELS_COLS]

In [ ]:
moral_model_cols_dict = {model: [f'{reason_col}_{moral_axis}_label'
  for moral_axis in keys.MORAL_AXES]
  for model, reason_col in zip(keys.MODELS, keys.REASON_COLS)}

In [ ]:
moral_labels_df.sum(axis=1).mean()

In [ ]:
moral_labels_df.mean(axis=0)

In [ ]:
df.rename({'harm_prob': 'harms_prob', 'harm_label': 'harms_label'}, axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

jitter = np.linspace(-0.35, 0.35, len(keys.MODELS))
width = jitter[1] - jitter[0]
ax.set_xlim([-0.7, 5.7])
for idx, moral_axis in enumerate(keys.MORAL_AXES):
    cols = [f'{model}_{moral_axis}_label' for model in keys.REASON_COLS]
    ax.bar(idx + jitter, df[cols].mean(axis=0).values,
           color=[f'C{idx}' for idx in range(len(keys.MODELS))],
           width=width)
    xmin = idx + jitter[0]
    xmax = idx + jitter[-1]
    x_min_lim, x_max_lim = ax.get_xlim()
    xmin_frac = (xmin - x_min_lim - width) / (x_max_lim - x_min_lim)
    xmax_frac = (xmax - x_min_lim + width) / (x_max_lim - x_min_lim)
    ax.axhline(y=df[f'{moral_axis}_label'].mean(),
               xmin=xmin_frac,
               xmax=xmax_frac,
               color='black',
               linestyle='--')
    
ax.set_ylim([0, 1])
ax.grid(axis='y')
ax.set_axisbelow(True)
ax.tick_params(axis='y', labelsize=13)
ax.set_xticks(np.arange(len(keys.MORAL_AXES)))
xtick_labels = keys.MORAL_AXES_LABELS_PLOT
xtick_labels[4] = 'Relational\nObligation'
xtick_labels[5] = 'Social\nNorms'
ax.set_xticklabels(bold_text(keys.MORAL_AXES_LABELS_PLOT), fontsize=13)

for idx, (model, model_label) in enumerate(zip(keys.MODELS, keys.MODEL_LABELS_PLOT)):
    ax.bar(5, 0, color=f'C{idx}', label=bold_text(model_label))

fig.legend(loc='center left',
           bbox_to_anchor=(0.1, 0.73),
           prop={'size': 10},
           ncol=1)

ax.set_ylabel(bold_text('Fraction of Samples'), fontsize=15)

plt.tight_layout()

plt.savefig('fig5_moral_themes.pdf', bbox_inches='tight')

In [ ]:
for model, moral_model_col in moral_model_cols_dict.items():
    print(model, df[moral_model_col].mean(axis=0))

In [ ]:
for model, moral_model_col in moral_model_cols_dict.items():
    print(model, df[moral_model_col].sum(axis=1).mean())

In [ ]:
for model, moral_model_col in moral_model_cols_dict.items():
    mean_correct = (df[moral_model_col].values == moral_labels_df.values).sum(axis=1).mean()
    print(model, mean_correct)